In [2]:
from proteusPy import (
    DisulfideList,
    Disulfide,
    Load_PDB_SS,
    load_disulfides_from_id,
    prune_extra_ss,
    check_header_from_file,
)

from proteusPy.logger_config import get_logger
from proteusPy.ssparser import (
    extract_ssbonds_and_atoms,
    print_disulfide_bond_info_dict,
    get_phipsi_atoms,
)
from proteusPy import calc_angle, calc_dihedral
import numpy as np


_logger = get_logger("testing")

In [3]:
from proteusPy import MODEL_DIR
from proteusPy.logger_config import get_logger
from proteusPy import DisulfideList
from proteusPy.ssparser import extract_ssbonds_and_atoms
import logging

_logger = get_logger("Testing")


def Nload_disulfides_from_id(
    pdb_id: str,
    pdb_dir=MODEL_DIR,
    model_numb=0,
    verbose=False,
    quiet=True,
    dbg=False,
    cutoff=-1.0,
) -> DisulfideList:
    """
    Loads the Disulfides by PDB ID and returns a ```DisulfideList``` of Disulfide objects.
    Assumes the file is downloaded in the pdb_dir path.

    *NB:* Requires EGS-Modified BIO.parse_pdb_header.py from https://github.com/suchanek/biopython

    :param pdb_id: the name of the PDB entry.
    :param pdb_dir: path to the PDB files, defaults to MODEL_DIR - this is: PDB_DIR/good and are
    the pre-parsed PDB files that have been scanned by the DisulfideDownloader program.
    :param model_numb: model number to use, defaults to 0 for single structure files.
    :param verbose: print info while parsing
    :return: a list of Disulfide objects initialized from the file.

    Example:

    PDB_DIR defaults to os.getenv('PDB').
    To load the Disulfides from the PDB ID 5rsa we'd use the following:

    >>> from proteusPy import DisulfideList, load_disulfides_from_id
    >>> from proteusPy.ProteusGlobals import DATA_DIR
    >>> SSlist = DisulfideList([],'5rsa')
    >>> SSlist = load_disulfides_from_id('5rsa', pdb_dir=DATA_DIR, verbose=False)
    >>> SSlist
    [<Disulfide 5rsa_26A_84A, Source: 5rsa, Resolution: 2.0 Å>, <Disulfide 5rsa_40A_95A, Source: 5rsa, Resolution: 2.0 Å>, <Disulfide 5rsa_58A_110A, Source: 5rsa, Resolution: 2.0 Å>, <Disulfide 5rsa_65A_72A, Source: 5rsa, Resolution: 2.0 Å>]
    """
    import copy
    import os
    import warnings

    from proteusPy import Disulfide, extract_ssbonds_and_atoms, DisulfideList
    from proteusPy.ssparser import initialize_disulfide_from_coords

    i = 1
    proximal = distal = -1
    chain1_id = chain2_id = ""

    resolution = -1.0

    structure_fname = os.path.join(pdb_dir, f"pdb{pdb_id}.ent")
    # model = structure[model_numb]

    if dbg:
        print(f"-> load_disulfide_from_id() - Parsing structure: {pdb_id}:")

    SSList = DisulfideList([], pdb_id, resolution)

    # list of tuples with (proximal distal chaina chainb)
    # ssbonds = parse_ssbond_header_rec(ssbond_dict)

    ssbond_atom_list, num_ssbonds, errors = extract_ssbonds_and_atoms(
        structure_fname, verbose=verbose
    )

    if verbose:
        print(
            f"-> load_disulfides_from_id(): {pdb_id} has {num_ssbonds} SSBonds, found: {errors} errors"
        )

    # with warnings.catch_warnings():
    if quiet:
        _logger.setLevel(logging.ERROR)

    for pair in ssbond_atom_list["pairs"]:
        proximal, chain1_id = pair["proximal"]
        distal, chain2_id = pair["distal"]

        if not proximal.isnumeric() or not distal.isnumeric():
            mess = f" -> load_disulfides_from_id(): Cannot parse SSBond record (non-numeric IDs):\
            {pdb_id} Prox: {proximal} {chain1_id} Dist: {distal} {chain2_id}, ignoring."
            _logger.error(mess)
            continue
        else:
            proximal = int(proximal)
            distal = int(distal)

        if proximal == distal:
            if verbose:
                mess = f" -> load_disulfides_from_id(): SSBond record has (proximal == distal):\
                {pdb_id} Prox: {proximal} {chain1_id} Dist: {distal} {chain2_id}."
                _logger.info(mess)

        if verbose:
            _logger.info(
                f"SSBond: {i}: {pdb_id}: {proximal} {chain1_id} - {distal} {chain2_id}"
            )

        new_ss = initialize_disulfide_from_coords(
            ssbond_atom_list,
            pdb_id,
            chain1_id,
            chain2_id,
            resolution,
            quiet=quiet,
        )
        if new_ss is not None:
            SSList.append(new_ss)
        i += 1

    if quiet:
        _logger.setLevel(logging.WARNING)

    if cutoff > 0:
        SSList = SSList.filter_by_distance(cutoff)

    return copy.deepcopy(SSList)